### FINA 4380 with Marius Popescu

## Stock Returns

In [1]:
import numpy as np
import pandas as pd

In [13]:
rets_all = pd.read_csv('month_rets.csv',
                       index_col = 'date',
                       usecols = ['date','TICKER', 'PRC', 'CFACPR'],
                       parse_dates = True)

rets_aapl = rets_all[rets_all['TICKER']=='AAPL'].copy()
rets_aapl.head(3)

,TICKER,PRC,CFACPR
date,,,
2013-01-31,AAPL,455.48999,7.0
2013-02-28,AAPL,441.39999,7.0
2013-03-28,AAPL,442.66000,7.0


In [14]:
rets_aapl['ADJ_PRC'] = rets_aapl['PRC']/rets_aapl['CFACPR']
rets_aapl.head(3)

,TICKER,PRC,CFACPR,ADJ_PRC
date,,,,
2013-01-31,AAPL,455.48999,7.0,65.069999
2013-02-28,AAPL,441.39999,7.0,63.057141
2013-03-28,AAPL,442.66000,7.0,63.237143


In [15]:
rets_aapl.drop(['PRC','CFACPR'],axis=1,inplace=True)
rets_aapl.head(3)

,TICKER,ADJ_PRC
date,,
2013-01-31,AAPL,65.069999
2013-02-28,AAPL,63.057141
2013-03-28,AAPL,63.237143


#### Shifting Time Series Data 
**Shifting** refers to moving data backward and forward through time. Both Series and DataFrame have a `.shift()` method we can use. A common application for shifting time series financial data is computing the rate of return.

#### Shifting data forward

In [16]:
rets_aapl['previous_ADJ_PRC'] = rets_aapl['ADJ_PRC'].shift(1)

In [17]:
# By shifting forward, we lose the last row of data in the original DataFrame
rets_aapl.tail(3)

,TICKER,ADJ_PRC,previous_ADJ_PRC
date,,,
2017-10-31,AAPL,169.03999,154.12000
2017-11-30,AAPL,171.85001,169.03999
2017-12-29,AAPL,169.23000,171.85001


In [18]:
rets_aapl.head(3)

,TICKER,ADJ_PRC,previous_ADJ_PRC
date,,,
2013-01-31,AAPL,65.069999,NaN
2013-02-28,AAPL,63.057141,65.069999
2013-03-28,AAPL,63.237143,63.057141


#### Shifting data backwards

In [19]:
rets_aapl['ADJ_PRC_next'] = rets_aapl['ADJ_PRC'].shift(-1)

In [20]:
# By shifting backwards, we lose the first row of data in the original DataFrame
rets_aapl.head(3)

,TICKER,ADJ_PRC,previous_ADJ_PRC,ADJ_PRC_next
date,,,,
2013-01-31,AAPL,65.069999,NaN,63.057141
2013-02-28,AAPL,63.057141,65.069999,63.237143
2013-03-28,AAPL,63.237143,63.057141,63.254286


In [21]:
rets_aapl.tail(3)

,TICKER,ADJ_PRC,previous_ADJ_PRC,ADJ_PRC_next
date,,,,
2017-10-31,AAPL,169.03999,154.12000,171.85001
2017-11-30,AAPL,171.85001,169.03999,169.23000
2017-12-29,AAPL,169.23000,171.85001,NaN


In [22]:
rets_aapl.drop(['previous_ADJ_PRC','ADJ_PRC_next'],axis=1,inplace=True)
rets_aapl.head(3)

,TICKER,ADJ_PRC
date,,
2013-01-31,AAPL,65.069999
2013-02-28,AAPL,63.057141
2013-03-28,AAPL,63.237143


### 1. Simple Rate of Return

#### A. The first method

In [24]:
#rets_aapl['RET1'] = (rets_aapl['ADJ_PRC']/rets_aapl['ADJ_PRC'].shift(1))-1
rets_aapl['RET1'] = rets_aapl['ADJ_PRC'].div(rets_aapl['ADJ_PRC'].shift(1)).sub(1)
rets_aapl.head(3)

,TICKER,ADJ_PRC,RET1
date,,,
2013-01-31,AAPL,65.069999,NaN
2013-02-28,AAPL,63.057141,-0.030934
2013-03-28,AAPL,63.237143,0.002855


#### B. The second method

In [25]:
rets_aapl['RET2'] = rets_aapl['ADJ_PRC'].pct_change()
rets_aapl[['RET1', 'RET2']].head(3)

,RET1,RET2
date,,
2013-01-31,NaN,NaN
2013-02-28,-0.030934,-0.030934
2013-03-28,0.002855,0.002855


### 2. Log Return (Continously Compounded Rate of Return)

In [26]:
#rets_aapl['LOG_RET1'] = np.log(rets_aapl['ADJ_PRC']/rets_aapl['ADJ_PRC'].shift(1))
rets_aapl['LOG_RET1'] = np.log(rets_aapl['ADJ_PRC'].div(rets_aapl['ADJ_PRC'].shift(1)))
rets_aapl[['RET1', 'LOG_RET1']].head(3)

,RET1,LOG_RET1
date,,
2013-01-31,NaN,NaN
2013-02-28,-0.030934,-0.031422
2013-03-28,0.002855,0.002851


In [27]:
#rets_aapl['LOG_RET2'] = np.log(1+rets_aapl['ADJ_PRC'].pct_change())
rets_aapl['LOG_RET2'] = np.log(rets_aapl['ADJ_PRC'].pct_change().add(1))
rets_aapl[['RET2', 'LOG_RET2']].head(3)

,RET2,LOG_RET2
date,,
2013-01-31,NaN,NaN
2013-02-28,-0.030934,-0.031422
2013-03-28,0.002855,0.002851


### 3. Aggregating/Cumulating/Compounding Return

In [28]:
rets_aapl.drop(['TICKER','ADJ_PRC','RET2','LOG_RET2'],axis=1,inplace=True)
rets_aapl.rename({'RET1':'RET', 'LOG_RET1':'LOG_RET'},axis=1,inplace=True)
rets_aapl.head(3)     

,RET,LOG_RET
date,,
2013-01-31,NaN,NaN
2013-02-28,-0.030934,-0.031422
2013-03-28,0.002855,0.002851


#### A. Simple Return

In [48]:
# What are the quarterly returns?
quarter_ret = rets_aapl[['RET']].resample('Q').apply(lambda x: x.add(1).prod().sub(1))
quarter_ret.head(3)

,RET
date,
2013-03-31,-0.028167
2013-06-30,-0.104211
2013-09-30,0.202305


In [50]:
# What are the annual returns?
annual_ret = rets_aapl[['RET']].resample('A').apply(lambda x: x.add(1).prod().sub(1))
annual_ret.head(3)

,RET
date,
2013-12-31,0.231685
2014-12-31,0.377241
2015-12-31,-0.046385


In [57]:
# What is the 5-year cumulative (total) return?
rets_aapl[['RET']].add(1).prod().sub(1)

RET    1.600738
dtype: float64

In [41]:
# Cumulative return series
rets_aapl[['RET']].add(1).cumprod().sub(1).head(3)

,RET
date,
2013-01-31,NaN
2013-02-28,-0.030934
2013-03-28,-0.028167


#### B. Log Return

In [60]:
# What are the quarterly log returns?
quarter_logret = rets_aapl[['LOG_RET']].resample('Q').sum()
quarter_logret.head(3)

,LOG_RET
date,
2013-03-31,-0.028572
2013-06-30,-0.110050
2013-09-30,0.184241


In [61]:
# What are the annual log returns?
annual_logret = rets_aapl[['LOG_RET']].resample('A').sum()
annual_logret

,LOG_RET
date,
2013-12-31,0.208383
2014-12-31,0.320083
2015-12-31,-0.047495
2016-12-31,0.095604
2017-12-31,0.379221


In [59]:
# What is the 5-year cumulative (total) log return?
rets_aapl[['LOG_RET']].sum()

LOG_RET    0.955795
dtype: float64

In [204]:
# Cumulative return series
rets_aapl[['LOG_RET']].cumsum().head(3)

,LOG_RET
date,
2013-01-31,NaN
2013-02-28,-0.031422
2013-03-28,-0.028572
